In [1]:
pip install torch transformers nltk soundfile opencv-python-headless moviepy Pillow scipy

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install groq 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 4.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install moviepy numpy

Note: you may need to restart the kernel to use updated packages.


In [12]:
pip install nltk requests

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install moviepy pillow numpy

Note: you may need to restart the kernel to use updated packages.


In [3]:
from groq import Groq

# Initialize the Groq client with API key
client = Groq(api_key="your api key")

# Research paper summary in JSON format
research_summary = {
    "title": "Quantum Machine Learning: Bridging Computational Boundaries",
    "key_findings": [
        "Quantum algorithms can solve certain computational problems exponentially faster than classical algorithms.",
        "Quantum machine learning models demonstrate superior performance in complex pattern recognition tasks.",
        "Entanglement and superposition principles enable unprecedented computational capabilities."
    ],
    "methodology": [
        "Implemented quantum neural networks using 7-qubit quantum processors.",
        "Utilized variational quantum eigensolvers for optimization.",
        "Developed hybrid classical-quantum computation framework."
    ],
    "implications": [
        "Potential revolutionary impact on drug discovery processes.",
        "Enhanced cryptographic security mechanisms.",
        "Advanced optimization for complex financial modeling."
    ],
    "research_summary": (
        "Our groundbreaking research explores the intersection of quantum computing and machine learning, "
        "demonstrating unprecedented computational capabilities by leveraging quantum mechanical principles. "
        "By developing innovative quantum neural network architectures, we've unlocked new frontiers in "
        "computational efficiency and pattern recognition."
    )
}

# Voiceover script preparation steps
script_instructions = (
    "Step 1: Understanding the Summary\n"
    "Read and analyze the provided research paper summary.\n"
    "Identify key points, findings, and contributions.\n"
    "Break down the content into short, engaging script lines (~100-150 words for a 1-minute speech).\n\n"
    "Step 2: Script Generation\n"
    "Convert the summary into a compelling, concise voiceover script.\n"
    "Ensure smooth transitions between ideas to maintain viewer engagement.\n"
    "Generating script for the following research paper content:"
)

# Generate script using Groq API
chat_completion = client.chat.completions.create(
    messages=[
        {"role": "user", "content": script_instructions + str(research_summary)}
    ],
    model="llama-3.3-70b-versatile",
    stream=False,
)

# Print the generated script
print(chat_completion.choices[0].message.content)


Here's a possible script based on the research paper summary:

"Imagine a world where computers can solve complex problems exponentially faster than today. Welcome to the realm of Quantum Machine Learning. Our research has shown that quantum algorithms can outperform classical ones, demonstrating superior pattern recognition capabilities. By harnessing the power of entanglement and superposition, we've developed innovative quantum neural networks that unlock new frontiers in computational efficiency. With potential implications in drug discovery, cryptographic security, and financial modeling, our findings could revolutionize industries and transform the way we live and work. Join us as we bridge the boundaries of computation and explore the limitless possibilities of Quantum Machine Learning." 

Note: The script is approximately 150 words and can be delivered within a 1-minute speech.


In [10]:
import requests
import json
from pathlib import Path

def get_available_voices(api_key):
    """
    Get list of available voices from ElevenLabs API.
    
    Args:
        api_key (str): ElevenLabs API key
    
    Returns:
        list: List of available voice IDs and names
    """
    try:
        voices_url = "https://api.elevenlabs.io/v1/voices"
        headers = {"xi-api-key": api_key}
        
        response = requests.get(voices_url, headers=headers)
        response.raise_for_status()
        
        voices = response.json().get("voices", [])
        return [(voice["voice_id"], voice["name"]) for voice in voices]
        
    except Exception as e:
        print(f"Error fetching voices: {str(e)}")
        return []

def generate_speech(text, api_key, voice_id=None, output_file="output.mp3"):
    """
    Generate speech using ElevenLabs API and save to a file.
    
    Args:
        text (str): Text to convert to speech
        api_key (str): ElevenLabs API key
        voice_id (str): Voice ID to use (if None, will use first available voice)
        output_file (str): Output file path (default: "output.mp3")
    
    Returns:
        bool: True if successful, False otherwise
    """
    try:
        # First, get available voices
        voices = get_available_voices(api_key)
        if not voices:
            print("Error: No voices available")
            return False
            
        # If no voice_id provided or if provided voice_id is invalid,
        # use the first available voice
        if not voice_id or not any(v[0] == voice_id for v in voices):
            voice_id = voices[0][0]
            print(f"Using voice ID: {voice_id}")
        
        # API endpoint
        api_url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
        
        # Request payload
        payload = {
            "text": text,
            "model_id": "eleven_monolingual_v1",
            "voice_settings": {
                "stability": 0.7,
                "similarity_boost": 0.9
            }
        }
        
        # Headers
        headers = {
            "xi-api-key": api_key,
            "Content-Type": "application/json",
            "Accept": "audio/mpeg"
        }
        
        # Make the API request
        response = requests.post(api_url, headers=headers, json=payload)
        
        # Check if request was successful
        response.raise_for_status()
        
        # Ensure the output directory exists
        output_path = Path(output_file)
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the audio file
        with open(output_file, "wb") as f:
            f.write(response.content)
            
        print(f"Speech synthesis complete! Audio saved to: {output_file}")
        return True
        
    except requests.exceptions.RequestException as e:
        if hasattr(e, 'response') and e.response is not None:
            if e.response.status_code == 401:
                print("Error: Invalid API key")
            elif e.response.status_code == 404:
                print(f"Error: Voice ID '{voice_id}' not found")
            else:
                try:
                    error_msg = e.response.json().get('detail', str(e))
                    print(f"API Error: {error_msg}")
                except:
                    print(f"Error during API request: {str(e)}")
        else:
            print(f"Connection error: {str(e)}")
        return False
        
    except Exception as e:
        print(f"Unexpected error: {str(e)}")
        return False

# Example usage:
if __name__ == "__main__":
    # Replace with your actual API key
    API_KEY = "api key"
    
    # Example text to convert
    text = chat_completion.choices[0].message.content
    
    # First, let's see what voices are available
    print("Available voices:")
    voices = get_available_voices(API_KEY)
    for voice_id, name in voices:
        print(f"- {name} (ID: {voice_id})")
    
    # Generate speech using the first available voice
    generate_speech(
        text=text,
        api_key=API_KEY,
        output_file="test_output.mp3"
    )

Available voices:
- Aria (ID: 9BWtsMINqrJLrRacOk9x)
- Roger (ID: CwhRBWXzGAHq8TQ4Fs17)
- Sarah (ID: EXAVITQu4vr4xnSDxMaL)
- Laura (ID: FGY2WhTYpPnrIDTdsKH5)
- Charlie (ID: IKne3meq5aSn9XLyUdCD)
- George (ID: JBFqnCBsd6RMkjVDRZzb)
- Callum (ID: N2lVS1w4EtoT3dr4eOWO)
- River (ID: SAz9YHcvj6GT2YYXdXww)
- Liam (ID: TX3LPaxmHKxFdv7VOQHJ)
- Charlotte (ID: XB0fDUnXU5powFXDhCwa)
- Alice (ID: Xb7hH8MSUJpSbSDYk0k2)
- Matilda (ID: XrExE9yKIg1WjnnlVkGX)
- Will (ID: bIHbv24MWmeRgasZH58o)
- Jessica (ID: cgSgspJ2msm6clMCkdW9)
- Eric (ID: cjVigY5qzO86Huf0OWal)
- Chris (ID: iP95p4xoKVk53GoZ742B)
- Brian (ID: nPczCjzI2devNBz1zQrb)
- Daniel (ID: onwK4e9ZLuTAKqWW03F9)
- Lily (ID: pFZP5JQG7iQjIQuC4Bku)
- Bill (ID: pqHfZKP75CvOlQylNhV4)
Using voice ID: 9BWtsMINqrJLrRacOk9x
Speech synthesis complete! Audio saved to: test_output.mp3


In [19]:
import os
import torch
import nltk
from collections import Counter
from transformers import pipeline
from diffusers import StableDiffusionXLPipeline, DPMSolverMultistepScheduler
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def setup_nltk():
    """Download required NLTK data."""
    try:
        nltk.data.find('tokenizers/punkt')
        nltk.data.find('corpora/stopwords')
    except LookupError:
        nltk.download('punkt')
        nltk.download('stopwords')
setup_nltk()

class TextToImageGenerator:
    def __init__(self, output_folder):
        self.output_folder = output_folder
        os.makedirs(output_folder, exist_ok=True)
        
        self.pipe = StableDiffusionXLPipeline.from_pretrained(
            "stabilityai/stable-diffusion-xl-base-1.0",
            torch_dtype=torch.float16,
            use_safetensors=True,
            variant="fp16"
        )
        
        self.pipe.scheduler = DPMSolverMultistepScheduler.from_config(
            self.pipe.scheduler.config,
            algorithm_type="dpmsolver++",
            use_karras_sigmas=True
        )
        
        if torch.cuda.is_available():
            self.pipe = self.pipe.to("cuda")
            self.pipe.enable_attention_slicing()
            self.pipe.enable_model_cpu_offload()
    
    def extract_keywords(self, text, top_n=10, min_length=4):
        """Extract meaningful keywords using TF-IDF and NLTK stopword filtering."""
        stop_words = set(stopwords.words('english'))
        additional_stops = {'using', 'used', 'like', 'also', 'would', 'could', 'may', 'might'}
        stop_words.update(additional_stops)
        
        tokens = word_tokenize(text.lower())
        filtered_words = [word for word in tokens if word.isalnum() and word not in stop_words and len(word) >= min_length]
        
        vectorizer = TfidfVectorizer(max_features=top_n, stop_words='english', ngram_range=(1, 2))
        try:
            tfidf_matrix = vectorizer.fit_transform([' '.join(filtered_words)])
            keywords = vectorizer.get_feature_names_out()
            print(f"🔍 Extracted Keywords: {keywords}")
            return list(keywords)
        except Exception as e:
            print(f"⚠️ Error extracting keywords: {e}")
            return []
    
    def generate_realistic_prompt(self, term):
        """Generate a photorealistic prompt for image generation."""
        base_prompts = [
            f"A stunning professional photograph of {term}, dramatic lighting, high-end photography, 8k resolution",
            f"Cinematic shot of {term}, perfect composition, golden hour lighting, ultra detailed, photorealistic",
            f"Breathtaking view of {term}, award-winning photography, atmospheric lighting, highly detailed",
            f"Professional studio photograph of {term}, expert lighting, ultra-sharp focus, magazine quality"
        ]
        
        import random
        prompt = random.choice(base_prompts)
        negative_prompt = ("cartoon, illustration, drawing, painting, sketches, low quality, low resolution, "
                         "blurry, text, watermark, signature, frame, deformed")
        
        return {"prompt": prompt, "negative_prompt": negative_prompt}
    
    def generate_image(self, term, idx):
        """Generate an image based on extracted keyword."""
        try:
            prompt_data = self.generate_realistic_prompt(term)
            print(f"\n🎨 Generating image for: {term}")
            print(f"📝 Using prompt: {prompt_data['prompt']}")
            
            image = self.pipe(
                prompt=prompt_data["prompt"],
                negative_prompt=prompt_data["negative_prompt"],
                num_inference_steps=50,
                guidance_scale=10,
                height=1024,
                width=1024,
                num_images_per_prompt=1
            ).images[0]
            
            output_path = os.path.join(self.output_folder, f"image_{idx}_generated.png")
            image.save(output_path)
            print(f"✅ Image saved to: {output_path}")
            return output_path
        except Exception as e:
            print(f"❌ Error generating image for {term}: {e}")
            return None

    def process_text(self, text):
        """Process direct text input, extract keywords, and generate images."""
        terms = self.extract_keywords(text)
        if not terms:
            print("⚠️ No keywords found. Skipping generation.")
            return []

        image_paths = []
        for idx, term in enumerate(terms[:10]):  # Ensure at most 10 images are generated
            image_path = self.generate_image(term, idx)
            if image_path:
                image_paths.append(image_path)

        return image_paths

# Example usage
if __name__ == "__main__":
    output_folder = "generated_images"
    generator = TextToImageGenerator(output_folder)
    
    sample_text = chat_completion.choices[0].message.content # Replace with your actual input
    image_paths = generator.process_text(sample_text)

    if image_paths:
        print(f"Generated image paths: {image_paths}")


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

🔍 Extracted Keywords: ['learning' 'machine' 'machine learning' 'quantum' 'quantum machine'
 'research' 'script' 'transform' 'transform live' 'unlock']

🎨 Generating image for: learning
📝 Using prompt: Cinematic shot of learning, perfect composition, golden hour lighting, ultra detailed, photorealistic


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Image saved to: generated_images/learning_0_generated.png

🎨 Generating image for: machine
📝 Using prompt: Cinematic shot of machine, perfect composition, golden hour lighting, ultra detailed, photorealistic


  0%|          | 0/50 [00:00<?, ?it/s]

...

✅ Image saved to: generated_images/transform_7_generated.png

🎨 Generating image for: transform live
📝 Using prompt: Cinematic shot of transform live, perfect composition, golden hour lighting, ultra detailed, photorealistic


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Image saved to: generated_images/transform live_8_generated.png

🎨 Generating image for: unlock
📝 Using prompt: Cinematic shot of unlock, perfect composition, golden hour lighting, ultra detailed, photorealistic


  0%|          | 0/50 [00:00<?, ?it/s]

✅ Image saved to: generated_images/unlock_9_generated.png
Generated image paths: ['generated_images/learning_0_generated.png', 'generated_images/machine_1_generated.png', 'generated_images/machine learning_2_generated.png', 'generated_images/quantum_3_generated.png', 'generated_images/quantum machine_4_generated.png', 'generated_images/research_5_generated.png', 'generated_images/script_6_generated.png', 'generated_images/transform_7_generated.png', 'generated_images/transform live_8_generated.png', 'generated_images/unlock_9_generated.png']


In [28]:
from moviepy.editor import *
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont

def create_simple_video(image_paths, audio_path, output_path="output_video.mp4"):
    """
    Create a video from static images and audio with correct PIL text rendering.
    """
    try:
        # Verify files exist
        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"Audio file not found: {audio_path}")
            
        existing_images = []
        for path in image_paths:
            if os.path.exists(path):
                existing_images.append(path)
            else:
                print(f"Warning: Image not found: {path}")
        
        if not existing_images:
            raise FileNotFoundError("No valid image files found")

        # Load audio
        audio = AudioFileClip(audio_path)
        
        # Calculate duration for each image
        duration_per_image = audio.duration / len(existing_images)
        
        # Create clips from images
        clips = []
        for idx, img_path in enumerate(existing_images):
            try:
                # Create image clip
                clip = ImageClip(img_path).set_duration(duration_per_image)
                
                # Resize to 1920x1080 while maintaining aspect ratio
                target_height = 1080
                target_width = 1920
                
                # Calculate new size maintaining aspect ratio
                aspect_ratio = clip.size[0] / clip.size[1]
                if aspect_ratio > (target_width / target_height):
                    new_width = target_width
                    new_height = int(target_width / aspect_ratio)
                else:
                    new_height = target_height
                    new_width = int(target_height * aspect_ratio)
                
                clip = clip.resize((new_width, new_height))
                
                # Create background
                bg = ColorClip((target_width, target_height), color=(0, 0, 0))
                bg = bg.set_duration(duration_per_image)
                
                # Position the image in the center
                x_center = (target_width - new_width) // 2
                y_center = (target_height - new_height) // 2
                clip = clip.set_position((x_center, y_center))
                
                # Add fade effects
                clip = clip.crossfadein(0.5).crossfadeout(0.5)
                
                # Create a simple colored rectangle for text background
                txt_bg = ColorClip(
                    (400, 60), 
                    color=(0, 0, 0)
                ).set_opacity(0.7).set_duration(duration_per_image)
                txt_bg = txt_bg.set_position(('center', 'bottom'))
                
                # Create composite without text first
                comp_clip = CompositeVideoClip([bg, clip, txt_bg], size=(target_width, target_height))
                
                # Draw text directly on the clip using PIL
                def draw_text(frame):
                    img = Image.fromarray(frame)
                    draw = ImageDraw.Draw(img)
                    text = f"Image {idx+1}"
                    font_size = 40
                    # Use default font with specified size
                    try:
                        font = ImageFont.truetype("/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf", font_size)
                    except:
                        font = ImageFont.load_default()
                    
                    # Get text size using font.getbbox()
                    bbox = font.getbbox(text)
                    text_width = bbox[2] - bbox[0]
                    text_height = bbox[3] - bbox[1]
                    
                    # Calculate position
                    x = (target_width - text_width) // 2
                    y = target_height - 80
                    
                    # Draw text
                    draw.text((x, y), text, fill=(255, 255, 255), font=font)
                    return np.array(img)
                
                # Apply text using PIL
                final_clip = comp_clip.fl_image(draw_text)
                clips.append(final_clip)
                
            except Exception as e:
                print(f"Error processing image {img_path}: {e}")
                continue

        # Concatenate all clips
        if not clips:
            raise ValueError("No clips were successfully created")
            
        final_video = concatenate_videoclips(clips)
        
        # Add audio
        final_video = final_video.set_audio(audio)
        
        # Write the final video
        final_video.write_videofile(
            output_path,
            fps=24,
            codec='libx264',
            audio_codec='aac',
            threads=4,
            preset='medium'
        )
        
        # Clean up
        audio.close()
        final_video.close()
        for clip in clips:
            clip.close()
        
        print(f"✅ Video successfully created: {output_path}")
        
    except Exception as e:
        print(f"❌ Error creating video: {e}")
        raise

# Example usage
if __name__ == "__main__":
    # Your image paths
    image_paths = [
        "/kaggle/working/generated_images/learning_0_generated.png",
        "/kaggle/working/generated_images/machine_1_generated.png",
        "/kaggle/working/generated_images/machine learning_2_generated.png",
        "/kaggle/working/generated_images/quantum_3_generated.png",
        "/kaggle/working/generated_images/quantum machine_4_generated.png",
        "/kaggle/working/generated_images/research_5_generated.png",
        "/kaggle/working/generated_images/script_6_generated.png",
        "/kaggle/working/generated_images/transform_7_generated.png",
        "/kaggle/working/generated_images/transform live_8_generated.png",
        "/kaggle/working/generated_images/unlock_9_generated.png"
    ]
    
    # Your audio path
    audio_path = "/kaggle/working/test_output.mp3"
    
    # Create the video
    create_simple_video(image_paths, audio_path)

Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video output_video.mp4



Moviepy - Done !
Moviepy - video ready output_video.mp4
✅ Video successfully created: output_video.mp4


In [42]:
import os
import numpy as np
from moviepy.editor import *
from PIL import Image, ImageDraw, ImageFont

def create_simple_video(image_paths, audio_path, output_path="output_video.mp4"):
    """
    Create a video from static images and audio with correct PIL text rendering.
    """
    try:
        # Verify files exist
        if not os.path.exists(audio_path):
            raise FileNotFoundError(f"Audio file not found: {audio_path}")
            
        existing_images = [path for path in image_paths if os.path.exists(path)]
        if not existing_images:
            raise FileNotFoundError("No valid image files found")

        # Load audio
        audio = AudioFileClip(audio_path)
        
        # Calculate duration for each image
        duration_per_image = audio.duration / len(existing_images)
        
        # Create clips from images
        clips = []
        for idx, img_path in enumerate(existing_images):
            try:
                # Create image clip
                clip = ImageClip(img_path).set_duration(duration_per_image)
                
                # Resize to 1920x1080 while maintaining aspect ratio
                clip = clip.resize(height=1080).resize(width=1920)
                
                # Create background
                bg = ColorClip((1920, 1080), color=(0, 0, 0)).set_duration(duration_per_image)
                
                # Position the image in the center
                clip = clip.set_position('center')
                
                # Add fade effects
                clip = clip.crossfadein(0.5).crossfadeout(0.5)
                
                # Create a simple colored rectangle for text background
                txt_bg = ColorClip((400, 60), color=(0, 0, 0)).set_opacity(0.7).set_duration(duration_per_image)
                txt_bg = txt_bg.set_position(('center', 'bottom'))
                
                # Create composite without text first
                comp_clip = CompositeVideoClip([bg, clip, txt_bg], size=(1920, 1080))
                
                # Draw text directly on the clip using PIL
                def draw_text(frame):
                    img = Image.fromarray(frame)
                    draw = ImageDraw.Draw(img)
                    text = f"Image {idx+1}"
                    font_size = 40
                    try:
                        font = ImageFont.truetype("arial.ttf", font_size)
                    except:
                        font = ImageFont.load_default()
                    bbox = draw.textbbox((0, 0), text, font=font)
                    text_width = bbox[2] - bbox[0]
                    text_height = bbox[3] - bbox[1]
                    # text_width, text_height = draw.textsize(text, font=font)
                    x = (1920 - text_width) // 2
                    y = 1080 - 80
                    
                    draw.text((x, y), text, fill=(255, 255, 255), font=font)
                    return np.array(img)
                
                # Apply text using PIL
                final_clip = comp_clip.fl_image(draw_text)
                clips.append(final_clip)
                
            except Exception as e:
                print(f"Error processing image {img_path}: {e}")
                continue

        # Concatenate all clips
        if not clips:
            raise ValueError("No clips were successfully created")
            
        final_video = concatenate_videoclips(clips)
        
        # Add audio
        final_video = final_video.set_audio(audio)
        
        # Write the final video
        final_video.write_videofile(
            output_path,
            fps=24,
            codec='libx264',
            audio_codec='aac',
            threads=4,
            preset='medium'
        )
        
        # Clean up
        audio.close()
        final_video.close()
        for clip in clips:
            clip.close()
        
        print(f"✅ Video successfully created: {output_path}")
        
    except Exception as e:
        print(f"❌ Error creating video: {e}")
        raise

# Example usage
if __name__ == "__main__":
    # Your image paths
    image_paths = [
        "/kaggle/working/generated_images/learning_0_generated.png",
        "/kaggle/working/generated_images/machine_1_generated.png",
        "/kaggle/working/generated_images/machine learning_2_generated.png",
        "/kaggle/working/generated_images/quantum_3_generated.png",
        "/kaggle/working/generated_images/quantum machine_4_generated.png",
        "/kaggle/working/generated_images/research_5_generated.png",
        "/kaggle/working/generated_images/script_6_generated.png",
        "/kaggle/working/generated_images/transform_7_generated.png",
        "/kaggle/working/generated_images/transform live_8_generated.png",
        "/kaggle/working/generated_images/unlock_9_generated.png"
    ]
    
    # Your audio path
    audio_path = "/kaggle/working/test_output.mp3"
    
    # Create the video
    create_simple_video(image_paths, audio_path)




                                                              


t:   1%|          | 50/8940 [22:23<54:54,  2.70it/s, now=None][A


                                                              


t:   1%|          | 50/8940 [22:23<54:54,  2.70it/s, now=None][A

Moviepy - Building video output_video.mp4.
MoviePy - Writing audio in output_videoTEMP_MPY_wvf_snd.mp4



chunk:  96%|█████████▋| 1152/1195 [00:01<00:00, 735.34it/s, now=None]
                                                                     


                                                              


t:   1%|          | 50/8940 [22:25<54:54,  2.70it/s, now=None][A


                                                              


t:   1%|          | 50/8940 [22:25<54:54,  2.70it/s, now=None][A

MoviePy - Done.
Moviepy - Writing video output_video.mp4




t: 100%|██████████| 1301/1301 [03:02<00:00,  7.17it/s, now=None]
                                                                


                                                              


t:   1%|          | 50/8940 [25:29<54:54,  2.70it/s, now=None][A


                                                              


t:   1%|          | 50/8940 [25:29<54:54,  2.70it/s, now=None][A

Moviepy - Done !
Moviepy - video ready output_video.mp4
✅ Video successfully created: output_video.mp4
